Importing Libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

Scraping Wikipedia data into a Data Frame

In [4]:
import requests 
from bs4 import BeautifulSoup

In [15]:
url = 'https://en.wikipedia.org/wiki/Category:Suburbs_of_Washington,_D.C.'
data = requests.get(url).text
soup = BeautifulSoup(data, 'html.parser')
soup


<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>Category:Suburbs of Washington, D.C. - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"86e0521d-b2fc-4da4-a440-d155b4da64cf","wgCSPNonce":!1,"wgCanonicalNamespace":"Category","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":14,"wgPageName":"Category:Suburbs_of_Washington,_D.C.","wgTitle":"Suburbs of Washington, D.C.","wgCurRevisionId":885301933,"wgRevisionId":885301933,"wgArticleId":60086016,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Geography of Washington, D.C.","Suburbs by city","Suburbs in Maryland","Suburbs in Virginia","Was

In [22]:
neighborhoods = []
for row in soup.find_all("div", class_="mw-category")[1].findAll("li"):
    neighborhoods.append(row.text)
neighborhoods

['Avondale, Maryland',
 'Brookmont, Maryland',
 'Capitol Heights, Maryland',
 'Chevy Chase (town), Maryland',
 'Chevy Chase Section Five, Maryland',
 'Chevy Chase Section Three, Maryland',
 'Chevy Chase Village, Maryland',
 'Chevy Chase (CDP), Maryland',
 'Chillum, Maryland',
 'Coral Hills, Maryland',
 'Forest Glen Park, Maryland',
 'Friendship Village, Maryland',
 "Martin's Additions, Maryland",
 'Seat Pleasant, Maryland',
 'Somerset, Maryland',
 'Takoma Park, Maryland']

In [23]:
dc_df = pd.DataFrame({"Neighborhood": neighborhoods})
dc_df.head()

,Neighborhood
0,"Avondale, Maryland"
1,"Brookmont, Maryland"
2,"Capitol Heights, Maryland"
3,"Chevy Chase (town), Maryland"
4,"Chevy Chase Section Five, Maryland"


In [24]:
dc_df.shape


(16, 1)

Let's get geographical coordinates

In [30]:
# define a function to get coordinates
def get_coords(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Washington DC, US'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [27]:
!pip install geocoder
import geocoder # to get coordinates
print('library installed')

library installed


In [33]:
# call the function to get the coordinates and store them in a new list
coords = [get_coords(neighborhood) for neighborhood in dc_df["Neighborhood"].tolist()]
coords

[[38.948500000000024, -76.97588999999994],
 [38.941790000000026, -77.12075999999996],
 [38.88533000000007, -76.91361999999998],
 [38.973558914981766, -77.09490743223016],
 [38.984290000000044, -77.07241999999997],
 [38.979750000000024, -77.07481999999999],
 [38.971160000000054, -77.07714999999996],
 [38.98249000000004, -77.08721999999995],
 [38.96411000000006, -76.99685999999997],
 [38.870400000000075, -76.92177999999996],
 [39.01173000000006, -77.06068999999997],
 [39.61280000000005, -77.76886999999994],
 [38.991495049088954, -77.04928992178723],
 [38.888090000000034, -76.89847999999995],
 [38.968740000000025, -77.09113999999994],
 [38.98126000000008, -77.00987999999995]]

In [35]:
#Create a temp dataframe
coords_df = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [36]:
# Add the coordinates into the main dataframe
dc_df['Latitude'] = coords_df['Latitude']
dc_df['Longitude'] = coords_df['Longitude']

In [38]:
dc_df

,Neighborhood,Latitude,Longitude
0,"Avondale, Maryland",38.948500,-76.975890
1,"Brookmont, Maryland",38.941790,-77.120760
2,"Capitol Heights, Maryland",38.885330,-76.913620
3,"Chevy Chase (town), Maryland",38.973559,-77.094907
4,"Chevy Chase Section Five, Maryland",38.984290,-77.072420
5,"Chevy Chase Section Three, Maryland",38.979750,-77.074820
6,"Chevy Chase Village, Maryland",38.971160,-77.077150
7,"Chevy Chase (CDP), Maryland",38.982490,-77.087220
8,"Chillum, Maryland",38.964110,-76.996860
9,"Coral Hills, Maryland",38.870400,-76.921780


Let's now create a map of Washington DC with these neighborhoods superimposed

In [40]:
city = 'Washington DC, US'

geolocator = Nominatim(user_agent="kuala_lumpur_explorer")
location = geolocator.geocode(city)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Washington, DC are {}, {}.'.format(latitude, longitude))

The coordinates of Washington, DC are 38.8949855, -77.0365708.


In [48]:
# create map of Washington DC using latitude and longitude values
map_dc = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(dc_df['Latitude'], dc_df['Longitude'], dc_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_dc)  
    
map_dc

Now let's use the Foursquare API to explore

In [49]:
# my credentials are below

CLIENT_ID = '5B2JWHSLBC1GMIJQL2WEUO3C2GB0F2NKHFC4R0NVRSOINQGG'
CLIENT_SECRET = 'CVSR55SZI2XCI4ESQXTPET3TE5I4ACQX4NCPQC3QE10R30HG'
VERSION = '20200715'

let's get the top 100 venues within a radius of 2000 meters

In [50]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(dc_df['Latitude'], dc_df['Longitude'], dc_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [51]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1118, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Avondale, Maryland",38.9485,-76.97589,Mid Atlantic Seafood,38.949829,-76.968403,Seafood Restaurant
1,"Avondale, Maryland",38.9485,-76.97589,Checkers,38.948970,-76.968544,Fast Food Restaurant
2,"Avondale, Maryland",38.9485,-76.97589,7-Eleven,38.944778,-76.972546,Convenience Store
3,"Avondale, Maryland",38.9485,-76.97589,PG Pool,38.948643,-76.963921,Pool
4,"Avondale, Maryland",38.9485,-76.97589,Q.C. Liquors,38.950845,-76.967792,Liquor Store


We can see how many venues were found for each neighborhood

In [52]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Avondale, Maryland",70,70,70,70,70,70
"Brookmont, Maryland",21,21,21,21,21,21
"Capitol Heights, Maryland",45,45,45,45,45,45
"Chevy Chase (CDP), Maryland",98,98,98,98,98,98
"Chevy Chase (town), Maryland",100,100,100,100,100,100
"Chevy Chase Section Five, Maryland",40,40,40,40,40,40
"Chevy Chase Section Three, Maryland",87,87,87,87,87,87
"Chevy Chase Village, Maryland",100,100,100,100,100,100
"Chillum, Maryland",96,96,96,96,96,96


In [53]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 175 uniques categories.


In [54]:
venues_df['VenueCategory'].unique()[:50]

array(['Seafood Restaurant', 'Fast Food Restaurant', 'Convenience Store',
       'Pool', 'Liquor Store', 'Video Game Store', 'Dance Studio', 'Café',
       'Grocery Store', 'Pizza Place', 'Ice Cream Shop',
       'Vegetarian / Vegan Restaurant', 'Recreation Center',
       'Chinese Restaurant', 'Donut Shop', 'Dog Run', 'Discount Store',
       'Gym Pool', 'Bank', 'Park', 'Latin American Restaurant',
       'Video Store', 'Vietnamese Restaurant', 'Storage Facility',
       'Southern / Soul Food Restaurant', 'Sandwich Place',
       'Fried Chicken Joint', 'Pharmacy', 'Gas Station', 'Bus Station',
       'Tennis Court', 'Bar', 'Plaza', 'Supermarket', 'Flower Shop',
       'Boarding House', 'Mediterranean Restaurant', "Men's Store",
       'Trail', 'Deli / Bodega', 'Japanese Restaurant', 'Bakery',
       'Coffee Shop', 'American Restaurant', 'Historic Site',
       'Border Crossing', 'Gourmet Shop', 'Stables', 'Church',
       'Gym / Fitness Center'], dtype=object)

In [59]:
#check if donut shop is amongst the categories
"Donut Shop" in venues_df['VenueCategory'].unique()

True

Let's now analyze each neighborhood

In [61]:
# one hot encoding
dc_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dc_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [dc_onehot.columns[-1]] + list(dc_onehot.columns[:-1])
dc_onehot = dc_onehot[fixed_columns]

dc_onehot.head()

,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Big Box Store,Bistro,Boarding House,Bookstore,Border Crossing,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chinese Restaurant,Church,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hospital,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Optical Shop,Outlet Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roller Rink,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Avondale, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Avondale, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Avondale, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Avondale, Maryland",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [63]:
#group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

dc_grouped = dc_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(dc_grouped.shape)
dc_grouped

(16, 176)


,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Arcade,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Big Box Store,Bistro,Boarding House,Bookstore,Border Crossing,Bowling Alley,Breakfast Spot,Brewery,Building,Burger Joint,Bus Station,Bus Stop,Café,Caribbean Restaurant,Chinese Restaurant,Church,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Electronics Store,Ethiopian Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Historic Site,History Museum,Hospital,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Music Store,Music Venue,Neighborhood,New American Restaurant,Noodle House,Office,Optical Shop,Outlet Store,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roller Rink,Salad Place,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Stables,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Avondale, Maryland",0.00000,0.000000,0.00000,0.00,0.000000,0.00,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.014286,0.014286,0.00000,0.000000,0.000000,0.000000,0.000000,0.014286,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.014286,0.000000,0.042857,0.000000,0.000000,0.000000,0.00000,0.000000,0.114286,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.000000,0.000000,0.000000,0.000000,0.042857,0.000000,0.014286,0.014286,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.014286,0.000000,0.00000,0.000,0.000000,0.014286,0.000000,0.00,0.00,0.042857,0.000000,0.00000,0.000000,0.000000,0.000000,0.071429,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.014286,0.000000,0.000000,0.071429,0.00,0.00000,0.00000,0.014286,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.057143,0.00,0.000000,0.00,0.000000,0.028571,0.00,0.028571,0.000000,0.014286,0.014286,0.000000,0.000000,0.000000,0.014286,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.014286,0.014286,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.00,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.014286,0.014286,0.000000,0.000000,0.00,0.000000,0.00,0.014286,0.00000,0.000000,0.000000,0.000000,0.000000,0.014286,0.01428

In [65]:
len(dc_grouped[dc_grouped["Donut Shop"] > 0])

6

In [68]:
#new data frame for only donut shops
dc_donut = dc_grouped[["Neighborhoods","Donut Shop"]]
dc_donut.head()

,Neighborhoods,Donut Shop
0,"Avondale, Maryland",0.014286
1,"Brookmont, Maryland",0.000000
2,"Capitol Heights, Maryland",0.022222
3,"Chevy Chase (CDP), Maryland",0.010204
4,"Chevy Chase (town), Maryland",0.000000


Clustering Neighborhoods

In [69]:
kclusters = 3

dc_clustering = dc_donut.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dc_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 1, 2, 0, 0, 0, 0, 2, 0], dtype=int32)

In [71]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
dc_merged = dc_donut.copy()

# add clustering labels
dc_merged["Cluster Labels"] = kmeans.labels_
dc_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
dc_merged.head()

,Neighborhood,Donut Shop,Cluster Labels
0,"Avondale, Maryland",0.014286,2
1,"Brookmont, Maryland",0.000000,0
2,"Capitol Heights, Maryland",0.022222,1
3,"Chevy Chase (CDP), Maryland",0.010204,2
4,"Chevy Chase (town), Maryland",0.000000,0


In [72]:
dc_merged = dc_merged.join(dc_df.set_index("Neighborhood"), on="Neighborhood")

print(dc_merged.shape)
dc_merged.head() # check the last columns!

(16, 5)


,Neighborhood,Donut Shop,Cluster Labels,Latitude,Longitude
0,"Avondale, Maryland",0.014286,2,38.948500,-76.975890
1,"Brookmont, Maryland",0.000000,0,38.941790,-77.120760
2,"Capitol Heights, Maryland",0.022222,1,38.885330,-76.913620
3,"Chevy Chase (CDP), Maryland",0.010204,2,38.982490,-77.087220
4,"Chevy Chase (town), Maryland",0.000000,0,38.973559,-77.094907


In [73]:
# sort the results by cluster labels
print(dc_merged.shape)
dc_merged.sort_values(["Cluster Labels"], inplace=True)
dc_merged

(16, 5)


,Neighborhood,Donut Shop,Cluster Labels,Latitude,Longitude
1,"Brookmont, Maryland",0.000000,0,38.941790,-77.120760
4,"Chevy Chase (town), Maryland",0.000000,0,38.973559,-77.094907
5,"Chevy Chase Section Five, Maryland",0.000000,0,38.984290,-77.072420
6,"Chevy Chase Section Three, Maryland",0.000000,0,38.979750,-77.074820
7,"Chevy Chase Village, Maryland",0.000000,0,38.971160,-77.077150
9,"Coral Hills, Maryland",0.000000,0,38.870400,-76.921780
10,"Forest Glen Park, Maryland",0.000000,0,39.011730,-77.060690
11,"Friendship Village, Maryland",0.000000,0,39.612800,-77.768870
12,"Martin's Additions, Maryland",0.000000,0,38.991495,-77.049290
14,"Somerset, Maryland",0.000000,0,38.968740,-77.091140


Finally let's visualize the clusters

In [76]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dc_merged['Latitude'], dc_merged['Longitude'], dc_merged['Neighborhood'], dc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Examining Clusters

In [80]:
#Cluster 1
dc_merged.loc[dc_merged['Cluster Labels'] == 0]

,Neighborhood,Donut Shop,Cluster Labels,Latitude,Longitude
1,"Brookmont, Maryland",0.0,0,38.941790,-77.120760
4,"Chevy Chase (town), Maryland",0.0,0,38.973559,-77.094907
5,"Chevy Chase Section Five, Maryland",0.0,0,38.984290,-77.072420
6,"Chevy Chase Section Three, Maryland",0.0,0,38.979750,-77.074820
7,"Chevy Chase Village, Maryland",0.0,0,38.971160,-77.077150
9,"Coral Hills, Maryland",0.0,0,38.870400,-76.921780
10,"Forest Glen Park, Maryland",0.0,0,39.011730,-77.060690
11,"Friendship Village, Maryland",0.0,0,39.612800,-77.768870
12,"Martin's Additions, Maryland",0.0,0,38.991495,-77.049290
14,"Somerset, Maryland",0.0,0,38.968740,-77.091140


In [79]:
#Cluster 2
dc_merged.loc[dc_merged['Cluster Labels'] == 1]

,Neighborhood,Donut Shop,Cluster Labels,Latitude,Longitude
2,"Capitol Heights, Maryland",0.022222,1,38.88533,-76.91362
13,"Seat Pleasant, Maryland",0.032258,1,38.88809,-76.89848


In [81]:
#Cluster 3
dc_merged.loc[dc_merged['Cluster Labels'] == 2]

,Neighborhood,Donut Shop,Cluster Labels,Latitude,Longitude
0,"Avondale, Maryland",0.014286,2,38.94850,-76.97589
3,"Chevy Chase (CDP), Maryland",0.010204,2,38.98249,-77.08722
8,"Chillum, Maryland",0.010417,2,38.96411,-76.99686
15,"Takoma Park, Maryland",0.010000,2,38.98126,-77.00988
